In [ ]:
import os
import pathlib
import glob

In [ ]:
# analyse error
workdir = 

# does a NEW Local Error log file exists
LocalErrorFile = os.path.join(workdir, 'LocalError-0.log')
LocalErrorExists = os.path.isfile(LocalErrorFile)
fname = pathlib.Path(LocalErrorFile)
LocalError_ct = fname.stat().st_mtime
LocalErrorNew = 
error_files_ct.append((, fname.stat().st_size))

if :
    analyse_LocalError

In [ ]:
def get_last_errorfile(workdir):
    error_files = glob.glob(os.path.join(workdir, 'error.*'))
    error_files.extend(glob.glob(os.path.join(workdir, 'slurm*')))
    error_files_ct = []
    for e in error_files:
        fname = pathlib.Path(e)
        error_files_ct.append((fname.stat().st_mtime, fname.name))
    error_files_ct.sort()
    lastfile = os.path.join(workdir, error_files_ct[-1][1])
    return(lastfile)

In [ ]:
def error_empty(workdir):
    error_files = glob.glob(os.path.join(workdir, 'error.*'))
    error_files.extend(glob.glob(os.path.join(workdir, 'slurm*')))
    error_files_ct = []
    for e in error_files:
        fname = pathlib.Path(e)
        error_files_ct.append((fname.stat().st_mtime, fname.stat().st_size))
    error_files_ct.sort()
    is_empty = bool(error_files_ct[-1][1] == 0)
    return(is_empty)

In [10]:
error_message_lookup = {'e000': 'Unknwon error','e001':'Program received signal SIGSEGV: Segmentation fault - invalid memory reference.', 'e002':'Force Terminated job', 'e003':'module: command not found'}
def parse_error_file(file_content):
    for line in file_content:
        for k in error_message_lookup.keys():
            if error_message_lookup[k] in line:
                return(k)
    return('e000')

### Check if error file is not empty

In [ ]:
dirs = []
with open('/scicore/home/lilienfeld/sahre0000/projects/atomic_energies/results/slice_ve38/broken', 'r') as f:
    for line in f:
        dirs.append(line.strip('\n'))

In [ ]:
empty = []
not_empty = []
for d in dirs:
    if error_empty(d):
        empty.append(d)
    else:
        not_empty.append(d)

In [ ]:
empty

In [ ]:
not_empty

## Parse error file for specific error messages

### Force terminated

In [ ]:
error_dictionary = {'e000':[], 'e001':[], 'e002':[], 'e003':[]}
for d in dirs:
    # read content of error file with latest creation date
    with open(get_last_errorfile(d), 'r') as f:
        error_file = f.readlines()
        error_number = parse_error_file(error_file)
        error_dictionary[error_number].append(d)
#         for line in f:
#             if 'Force Terminated job' in line:
#                 force_terminated.append(d)
#                 break
#             elif 'module: command not found' in line:
#                 module_not_found.append(d)

In [ ]:
force_terminated.sort()
force_terminated

In [ ]:
'/scicore/home/lilienfeld/sahre0000/projects/atomic_energies/results/slice_ve38/dsgdb9nsd_002141/run0/ve_30'
'/scicore/home/lilienfeld/sahre0000/projects/atomic_energies/results/slice_ve38/dsgdb9nsd_002141/run0/ve_38'
'/scicore/home/lilienfeld/sahre0000/projects/atomic_energies/results/slice_ve38/dsgdb9nsd_003705/run0/ve_8'

### Check if restart file is ok

In [ ]:
def error_empty(workdir):
    error_files = glob.glob(os.path.join(workdir, 'error.*'))
    error_files.extend(glob.glob(os.path.join(workdir, 'slurm*')))
    error_files_ct = []
    for e in error_files:
        fname = pathlib.Path(e)
        error_files_ct.append((fname.stat().st_mtime, fname.stat().st_size))
    error_files_ct.sort()
    is_empty = bool(error_files_ct[-1][1] == 0)
    return(is_empty)

def size_ok(rfile, s=373.76):
    fname = pathlib.Path(rfile)
    fsize = fname.stat().st_size/(1024*1024)
    return(bool(fsize > s))

In [ ]:
for p in force_terminated:
    rfile = os.path.join(p, 'RESTART.1')
    print(p, size_ok(rfile))

### fixed files

In [ ]:
with open('/scicore/home/lilienfeld/sahre0000/projects/atomic_energies/results/slice_ve38/ftf', 'w') as f:
    for line in force_terminated:
        f.write(line+'\n')

In [ ]:
a = 'abc'
a[:2]

# Find dumped core-file dirs

In [1]:
import os

In [2]:
dirs = []
basepath = '/scicore/home/lilienfeld/sahre0000/projects/atomic_energies/results/amons'
with open('/scicore/home/lilienfeld/sahre0000/projects/atomic_energies/results/amons/core_files', 'r') as f:
    for line in f:
        path = [basepath]
        comp_ve = line.split('/')[:2]
        path.extend(comp_ve)
        dirs.append('/'.join(path))
dirs = list(set(dirs))
dirs.sort()
#dirs

In [3]:
def modify_input(file, modify_option):
    if modify_option == 'initialize random':
        add_initialize_random(file)
    
def add_initialize_random(file):
    with open(file, 'r') as f:
        file_list = f.readlines()
    file_list.insert(2, '  INITIALIZE WAVEFUNCTION RANDOM\n')
    with open(file, 'w') as f:
        f.writelines(file_list)

In [5]:
for d in dirs[1:-2]:
    input_file = os.path.join(d, 'run.inp')
    modify_input(input_file, 'initialize random')

In [9]:
with open('/scicore/home/lilienfeld/sahre0000/projects/atomic_energies/results/amons/ini_random', 'w') as f:
    for l in dirs[:-2]:
        f.write(l+'\n')

In [6]:
dirs[100]

'/scicore/home/lilienfeld/sahre0000/projects/atomic_energies/results/amons/amon_000052/ve_10'